In [ ]:
import sys
sys.path.append("..")

import random
import math
import time
import json
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.decoder import *
from src.models.transform import *
from src.models.util import *
from experiments import datasets
from experiments.denoise.resconv import ResConv

def resize(img, scale: float, mode: VF.InterpolationMode = VF.InterpolationMode.NEAREST):
    return VF.resize(img, [max(1, int(s * scale)) for s in img.shape[-2:]], mode, antialias=False)

In [ ]:
kernel_size = []
padding = []
for i in range(11):
    t = i / (11 - 1)
    ks = 3 * (1. - t) + t * 9
    ks = int(ks / 2) * 2 + 1
    kernel_size.append(ks)
    padding.append(int(math.floor(ks / 2)))
  
model = ResConv(
    in_channels=1,
    num_layers=11,
    channels=32,
    stride=1,
    kernel_size=kernel_size,
    padding=padding,
    activation="gelu",
    activation_last_layer="sigmoid",
    conv_groups=[1] + [1] * (11 - 2) + [1],
).eval()
state = torch.load("../checkpoints/denoise/resconv-group-hcrop-bw_ds-fmnist_l-11_ks1-3_ks2-9_ch-32_stride-1_groups-1_act-gelu/snapshot.pt")
print(f'inputs: {state["num_input_steps"]:,}')
model.load_state_dict(state["state_dict"])
print(f"params: {num_module_parameters(model):,}")

In [ ]:
fmnist = datasets.fmnist_dataset(shape=(1, 28, 28), train=False)
patches = next(iter(DataLoader(fmnist, batch_size=8*8)))[0]
VF.to_pil_image(make_grid(patches))

In [ ]:
cropped_patches = RandomCropHalfImage()(patches)
grid = make_grid(cropped_patches, padding=28)[:1]
VF.to_pil_image(grid)

In [ ]:
with torch.no_grad():
    r_grid = model(grid.unsqueeze(0))[0]
VF.to_pil_image(r_grid)

In [ ]:
image = VF.to_tensor(PIL.Image.open(
    #"/home/bergi/Pictures/diffusion/cthulhu-06.jpeg"
    "/home/bergi/Pictures/__diverse/HourSlack-EarsBleed-2-72SM.jpg"
    #"/home/bergi/Pictures/__diverse/2000_subgenius_bobco_primer.jpg"
    #"/home/bergi/Pictures/__diverse/longtime.png"
).convert("L"))
VF.to_pil_image(image)


In [ ]:
with torch.no_grad():
    display(VF.to_pil_image(model(1.-image.unsqueeze(0))[0]))

In [ ]:
def perspective_transform(
        image: torch.Tensor, 
        top: float = 1.,
        left: float = 1.,
        bottom: float = 1.,
        right: float = 1.,
):
    h, w = image.shape[-2:]
    top = max(-w // 2 + 1, (top - 1.) * w / 2)
    bottom = max(-w // 2 + 1, (bottom - 1.) * w / 2)
    left = max(-h // 2 + 1, (left - 1.) * h / 2)
    right = max(-h // 2 + 1, (right - 1.) * h / 2)
    return VF.perspective(
        image,
        [[0, 0], [w, 0], [w, h], [0, h]],
        [[-top, -left], [w + top, -right], [w + bottom, h + right], [-bottom, h + left]],
        interpolation=VF.InterpolationMode.BILINEAR,
        
    )

VF.to_pil_image(perspective_transform(
    image,
    top=.99,
    bottom=1.1,
    #left=1.5,
    #right=1.1,
))

In [ ]:
VF.affine?